In [1]:
%%capture
%pip install git+https://gitlab-ce.santanderbr.corp/infra_cfo/cfotools.git polars

In [2]:
from cfotools.tool import *
import polars as pl
import variaveis as var
import funcoes_varejo as func
import funcoes_pnl_anl as pnl_anl

In [ ]:
from importlib import reload
from warnings import simplefilter

simplefilter('ignore')
reload(var)
reload(func)
reload(pnl_anl)

In [ ]:
tb_analitica = pd.read_oracle('gc_clientes.tb_analitico_cenario_pnl_p27')
tb_analitica.columns = [col.upper() for col in tb_analitica.columns]
tb_analitica

In [ ]:
pivot_resultado = func.get_pivot_table(tb_analitica)
_pivot_resultado = pivot_resultado
# anos = pd.to_datetime(tb_analitica['MES'], format='%Y%m').dt.year.unique()
num_anos = len(pd.to_datetime(tb_analitica['MES'], format='%Y%m').dt.year.unique())

In [ ]:
_rmodelo_raw = func.gerar_tabela_raw('R_MODELO')
rmodelo_fin = func.gerar_tabela_rmodelo(func.gerar_tabela_raw('R_MODELO'), pivot_resultado, var.subtipos, var.segmentos, num_anos, var.cenario)

In [ ]:
for col in rmodelo_fin[0].columns:
    print(f'{col=}', rmodelo_fin[0][col].unique().tolist())

In [ ]:
rmodelo_raw = rmodelo_raw.iloc[1:]
rmodelo_raw = rmodelo_raw.drop(rmodelo_raw.iloc[:, 12:40], axis = 1)
rmodelo_raw = rmodelo_raw.rename(columns={'Unnamed: 9': '', '%': f'Reparto % {y1}', '%.1': f'Reparto % {y0}'})

In [ ]:
pmodelo_raw = func.gerar_tabela_raw('P_MODELO')
pmodelo_fin = func.gerar_tabela_pmodelo(pmodelo_raw, pivot_resultado, var.subtipos, var.segmentos, num_anos, var.cenario)

In [ ]:
pmodelop_fin = func.gerar_tabela_pmodelo_ponta(pmodelo_raw, pivot_resultado, var.subtipos, var.segmentos, num_anos, var.cenario)

In [ ]:
smodelo_fin = func.gerar_tabela_smodelo(pmodelo_fin, rmodelo_fin, func.gerar_tabela_raw('S_MODELO'), var.dic_dias_mes, var.dic_du_mes, var.s_anos, num_anos, var.cenario)

In [ ]:
cmodelo_fin = func.gerar_tabela_cascmodelo(func.gerar_tabela_raw('Casc_MODELO'), rmodelo_fin, num_anos, var.cenario)

In [ ]:
pmodelo_fin2, pmodelop_fin2, rmodelo_fin2, smodelo_fin2, cascmodelo_fin2 = func.ajustar_colunas_dfs(pmodelo_fin, pmodelop_fin, rmodelo_fin, smodelo_fin, cmodelo_fin, num_anos, var.cenario)
header_p, header_r, header_s, header_casc = func.gerar_headers_finais(tb_analitica, num_anos, var.cenario)
pmodelo_fin3, pmodelop_fin3, rmodelo_fin3, smodelo_fin3, cascmodelo_fin3 = func.ajustar_headers_finais(
    header_p, header_r, header_s, header_casc, pmodelo_fin2, pmodelop_fin2, rmodelo_fin2, smodelo_fin2, cascmodelo_fin2, num_anos, var.cenario
)
pmodelo_final, pmodelop_final, rmodelo_final, smodelo_final, cascmodelo_final = func.acrescentar_linhas_em_branco_renomear_colunas_data(
    pmodelo_fin3, pmodelop_fin3, rmodelo_fin3, smodelo_fin3, cascmodelo_fin3, num_anos, var.cenario
)

In [ ]:
df_pnl_VarTotal, df_pnl_Var = pnl_anl.gerar_df_pnl(cmodelo_fin, var.cenario, var.anomes_m0, var.anomes_m1)
df_pnl_Var.head(20)

In [ ]:
df_pnl_VarTotal.head(20)

In [ ]:
BAI_check, Segto_check, Segto_P_check, MoM_R01, MoM_R02, MoM_P01 = pnl_anl.gerar_df_analysis(
    rmodelo_fin, cmodelo_fin, pmodelo_fin, pmodelop_fin, var.cenario, var.segmentos, var.anomes_m0, var.anomes_m1, var.anomes_m2
)
rmodelo_pf_valid, rmodelo_pj_valid = pnl_anl.validar_exclusivos_pf_pj(rmodelo_fin2[6], rmodelo_fin2[11], var.cenario)
valid_contab_fict_zerados_y0 = pnl_anl.validar_contabil_ficticio_cod_zerados_y0(rmodelo_fin, var.cenario, num_anos)
valid_resultado_e_soma_contabil_ficticio = pnl_anl.validar_resultado_e_soma_contabil_ficticio(rmodelo_fin, var.cenario, num_anos)

In [ ]:
m0 = datetime.now().month
if m0 == 1 and var.cenario.lower() in ('avance', 'avanc', 'avan', 'fechto', 'fechamento', 'fecham'):
    y0 = datetime.now().year - 1
else:
    y0 = datetime.now().year

for a in range(len(valid_contab_fict_zerados_y0)):
    for b in valid_contab_fict_zerados_y0[a].columns:
        if f'{y0}' in b and (a not in (0,1,2,3,4)):
            print(f'segm: {a:02}  col: {b}')

In [ ]:
excel_valid = pnl_anl.gerar_excel_validadores(
    MoM_R01, MoM_R02, MoM_P01, rmodelo_pf_valid, rmodelo_pj_valid, valid_contab_fict_zerados_y0, valid_resultado_e_soma_contabil_ficticio, var.segmentos, var.cenario, var.mesano, var.versao
)

In [ ]:
if 'df_pnl_Var' not in globals():
    df_pnl_Var = pd.DataFrame()
if 'df_pnl_VarTotal' in globals():
    df_pnl_VarTotal = pd.DataFrame()

In [ ]:
path_excel_raw = func.gerar_excel_var_atual(df_pnl_VarTotal, df_pnl_Var, pmodelo_final, pmodelop_final, rmodelo_final, smodelo_final, cascmodelo_final, var.segmentos, var.cenario, var.mesano, var.versao)
func.personalizar_excel(path_excel_raw, var.cenario, var.mesano, var.versao, num_anos)